<h1 align=center><font size = 5>Clustering Toronto</font></h1>

## Introduction

In this lab, you will learn how to convert addresses into their equivalent latitude and longitude values. Also, you will use the Foursquare API to explore neighborhoods in Toronto. You will use the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. You will use the *k*-means clustering algorithm to complete this task. Finally, you will use the Folium library to visualize the neighborhoods in Toronto and their emerging clusters.

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests # library to handle requests

#!conda install -c conda-forge beautifulsoup4 --y
from bs4 import BeautifulSoup

#!conda install -c conda-forge lxml --y
from lxml import etree

from urllib.request import urlopen

print('Libraries imported.')

Libraries imported.


## 1. Download and Explore Dataset

We will use BeautifulSoup for Web Scraping. First of all we upload htlm page though requests

In [2]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
res

<Response [200]>

Response 200 means that the page downloaded successfully. We can proceed with Scraping

In [3]:
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table)) [0]
Postal_code = df["Postal code"].tolist()
Borough = df["Borough"].tolist()
Neighborhood = df["Neighborhood"].tolist()
print('Data downloaded!')
df.head()

Data downloaded!


,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


#### Replace '/' by comma for Neighborhoods in same Postal code area

In [4]:
df['Neighborhood'] = df['Neighborhood'].str.replace(' /',',')
df['Neighborhood'].head()

0                          NaN
1                          NaN
2                    Parkwoods
3             Victoria Village
4    Regent Park, Harbourfront
Name: Neighborhood, dtype: object

#### Replace all Null and 'Not assigned' values for Neighborhood with Borough

In [5]:
df=df.ffill(axis=1)
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned', df['Borough'],df['Neighborhood'])
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Drop all rows with 'Not assigned' value for Borough

In [6]:
Drop=df[df['Borough'] == 'Not assigned'].index
df.drop(Drop, inplace = True)
df.reset_index(drop=True, inplace=True)
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Let's check final shape of the dataframe

In [7]:
df.shape

(103, 3)

## 2. Load and Add the Location Data

#### Next, let's load the data.

In [8]:
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [9]:
geo_df = pd.read_csv('Geospatial_Coordinates.csv')
geo_df.sort_values(by=['Postal Code'], ignore_index = True)
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
geo_df.rename(columns={'Postal Code':'Postal code'}, inplace=True)
Toronto_df= pd.merge(df, geo_df, on='Postal code', how='right')
Toronto_df.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494



## 3. Explore and Cluster the Neighborhoods in Toronto

#### Define Foursquare Credentials and Version

In [60]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [14]:
# @hidden_cell
    CLIENT_ID = 'OHUPWIN5F4LBG0YBCXXL4JVJ5NOTVCIX0L5YESOQCVDW3CZF' # your Foursquare ID
    CLIENT_SECRET = 'NTRSBC3KTRORE3SNTPEC1JFRXSFL1JNHKKNQJKKLIBQYJDUJ' # your Foursquare Secret
    VERSION = '20180605' # Foursquare API version

In [20]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(Toronto_df['Borough'].unique()),
        Toronto_df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


#### Use geopy library to get the latitude and longitude values of New York City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [23]:
address = 'Toronto'

geolocator = Nominatim(user_agent="TO_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [29]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Borough'], Toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Central Toronto. So let's slice the original dataframe and create a new dataframe of the Central Toronto data.

In [42]:
Central_Toronto_data = Toronto_df[Toronto_df['Borough'] == 'Central Toronto'].reset_index(drop=True)
Central_Toronto_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678


Let's get the geographical coordinates of Manhattan.

In [31]:
address = 'Central Toronto, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.6534817, -79.3839347.


As we did with all of Toronto, let's visualizat Central Toronto the neighborhoods in it.

In [32]:
# create map of Manhattan using latitude and longitude values
map_ctoronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Central_Toronto_data['Latitude'], Central_Toronto_data['Longitude'], Central_Toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ctoronto)  
    
map_ctoronto

#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [33]:
Central_Toronto_data.loc[0, 'Neighborhood']

'Lawrence Park'

Get the neighborhood's latitude and longitude values.

In [16]:
neighborhood_latitude = Central_Toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Central_Toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Central_Toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


#### Now, let's get the top 100 venues that are in Lawrence Park within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [35]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea732e095feaf001b297e93'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 71,
  'suggestedBounds': {'ne': {'lat': 43.6579817045, 'lng': -79.37772678059432},
   'sw': {'lat': 43.6489816955, 'lng': -79.39014261940568}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          'lng'

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [36]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [37]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Eggspectation Bell Trinity Square,Breakfast Spot,43.653144,-79.381980
3,Indigo,Bookstore,43.653515,-79.380696
4,Chatime 日出茶太,Bubble Tea Shop,43.655542,-79.384684


And how many venues were returned by Foursquare?

In [38]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

71 venues were returned by Foursquare.


#### Let's create a function to repeat the same process to all the neighborhoods in Central Toronto

In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [40]:
# type your answer here

central_toronto_venues = getNearbyVenues(names=Central_Toronto_data['Neighborhood'],
                                   latitudes=Central_Toronto_data['Latitude'],
                                   longitudes=Central_Toronto_data['Longitude']
                                  )



Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
North Toronto West
The Annex, North Midtown, Yorkville
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park


#### Let's check the size of the resulting dataframe

In [43]:
print(central_toronto_venues.shape)
central_toronto_venues.head()

(113, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,Roselawn,43.711695,-79.416936,Rosalind's Garden Oasis,43.712189,-79.411978,Garden
4,Davisville North,43.712751,-79.390197,Homeway Restaurant & Brunch,43.712641,-79.391557,Breakfast Spot


Let's check how many venues were returned for each neighborhood

In [44]:
central_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,35,35,35,35,35,35
Davisville North,8,8,8,8,8,8
Forest Hill North & West,4,4,4,4,4,4
Lawrence Park,3,3,3,3,3,3
"Moore Park, Summerhill East",1,1,1,1,1,1
North Toronto West,22,22,22,22,22,22
Roselawn,1,1,1,1,1,1
"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park",16,16,16,16,16,16
"The Annex, North Midtown, Yorkville",23,23,23,23,23,23


#### Let's find out how many unique categories can be curated from all the returned venues

In [45]:
print('There are {} uniques categories.'.format(len(central_toronto_venues['Venue Category'].unique())))

There are 61 uniques categories.


### Analyze Each Neighborhood

In [46]:
# one hot encoding
Ctoronto_onehot = pd.get_dummies(central_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Ctoronto_onehot['Neighborhood'] = central_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Ctoronto_onehot.columns[-1]] + list(Ctoronto_onehot.columns[:-1])
Ctoronto_onehot = Ctoronto_onehot[fixed_columns]

Ctoronto_onehot.head()

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Bar,Breakfast Spot,Brewery,Burger Joint,Bus Line,...,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Roselawn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Davisville North,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [47]:
Ctoronto_onehot.shape

(113, 62)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [48]:
Ctoronto_grouped = Ctoronto_onehot.groupby('Neighborhood').mean().reset_index()
Ctoronto_grouped

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Bar,Breakfast Spot,Brewery,Burger Joint,Bus Line,...,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.028571,0.000000,0.000000,0.0000,0.028571,0.000,0.028571,0.000000,0.000000,...,0.0000,0.0000,0.057143,0.000000,0.028571,0.028571,0.00,0.000000,0.0000,0.000000
1,Davisville North,0.000000,0.000000,0.000000,0.0000,0.000000,0.125,0.000000,0.000000,0.000000,...,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.000000
2,Forest Hill North & West,0.000000,0.000000,0.000000,0.0000,0.000000,0.000,0.000000,0.000000,0.000000,...,0.0000,0.0000,0.250000,0.000000,0.000000,0.000000,0.25,0.000000,0.0000,0.000000
3,Lawrence Park,0.000000,0.000000,0.000000,0.0000,0.000000,0.000,0.000000,0.000000,0.333333,...,0.0000,0.0000,0.000000,0.333333,0.000000,0.000000,0.00,0.000000,0.0000,0.000000
4,"Moore Park, Summerhill East",0.000000,0.000000,0.000000,0.0000,0.000000,0.000,0.000000,0.000000,0.000000,...,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.000000
5,North Toronto West,0.000000,0.000000,0.045455,0.0000,0.000000,0.000,0.000000,0.000000,0.000000,...,0.0000,0.0000,0.000000,0.000000,0.000000,0.045455,0.00,0.000000,0.0000,0.045455
6,Roselawn,0.000000,0.000000,0.000000,0.0000,0.000000,0.000,0.000000,0.000000,0.000000,...,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.000000
7,"Summerhill West, Rathnelly, South Hill, Forest...",0.062500,0.000000,0.062500,0.0625,0.000000,0.000,0.000000,0.000000,0.000000,...,0.0625,0.0625,0.062500,0.000000,0.000000,0.000000,0.00,0.000000,0.0625,0.000000
8,"The Annex, North Midtown, Yorkville",0.043478,0.043478,0.000000,0.0000,0.000000,0.000,0.000000,0.043478,0.000000,...,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.043478,0.0000,0.000000


#### Let's confirm the new size

In [49]:
Ctoronto_grouped.shape

(9, 62)

#### Let's print each neighborhood along with the top 5 most common venues

In [50]:
num_top_venues = 5

for hood in Ctoronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Ctoronto_grouped[Ctoronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0         Pizza Place  0.09
1      Sandwich Place  0.09
2        Dessert Shop  0.09
3    Sushi Restaurant  0.06
4  Italian Restaurant  0.06


----Davisville North----
               venue  freq
0   Department Store  0.12
1     Breakfast Spot  0.12
2       Dance Studio  0.12
3  Food & Drink Shop  0.12
4               Park  0.12


----Forest Hill North & West----
                 venue  freq
0        Jewelry Store  0.25
1                Trail  0.25
2   Mexican Restaurant  0.25
3     Sushi Restaurant  0.25
4  American Restaurant  0.00


----Lawrence Park----
                 venue  freq
0          Swim School  0.33
1             Bus Line  0.33
2                 Park  0.33
3  American Restaurant  0.00
4           Restaurant  0.00


----Moore Park, Summerhill East----
                 venue  freq
0           Playground   1.0
1  American Restaurant   0.0
2                Hotel   0.0
3   Italian Restaurant   0.0
4        Jewelry Store   0.0


---

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [51]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [54]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Ctoronto_grouped['Neighborhood']

for ind in np.arange(Ctoronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Ctoronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Sushi Restaurant,Coffee Shop,Italian Restaurant,Café,Gym,Pharmacy,Greek Restaurant
1,Davisville North,Dance Studio,Sandwich Place,Hotel,Food & Drink Shop,Park,Department Store,Gym,Breakfast Spot,Brewery,Diner
2,Forest Hill North & West,Jewelry Store,Trail,Sushi Restaurant,Mexican Restaurant,Farmers Market,Department Store,Dessert Shop,Diner,Donut Shop,Yoga Studio
3,Lawrence Park,Swim School,Bus Line,Park,Yoga Studio,Deli / Bodega,Gas Station,Garden,Fried Chicken Joint,Food & Drink Shop,Flower Shop
4,"Moore Park, Summerhill East",Playground,Yoga Studio,Deli / Bodega,Gourmet Shop,Gas Station,Garden,Fried Chicken Joint,Food & Drink Shop,Flower Shop,Fast Food Restaurant


<a id='item4'></a>

### Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [57]:
# set number of clusters
kclusters = 5

Ctoronto_grouped_clustering = Ctoronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Ctoronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 4, 3, 2, 0, 1, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [58]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Ctoronto_merged = Central_Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Ctoronto_merged = Ctoronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Ctoronto_merged.head() # check the last columns!

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Swim School,Bus Line,Park,Yoga Studio,Deli / Bodega,Gas Station,Garden,Fried Chicken Joint,Food & Drink Shop,Flower Shop
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936,1,Garden,Yoga Studio,Gym,Gourmet Shop,Gas Station,Fried Chicken Joint,Food & Drink Shop,Flower Shop,Fast Food Restaurant,Farmers Market
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Dance Studio,Sandwich Place,Hotel,Food & Drink Shop,Park,Department Store,Gym,Breakfast Spot,Brewery,Diner
3,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307,4,Jewelry Store,Trail,Sushi Restaurant,Mexican Restaurant,Farmers Market,Department Store,Dessert Shop,Diner,Donut Shop,Yoga Studio
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Clothing Store,Coffee Shop,Yoga Studio,Café,Fast Food Restaurant,Diner,Dessert Shop,Mexican Restaurant,Park,Rental Car Location


Finally, let's visualize the resulting clusters

In [61]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Ctoronto_merged['Latitude'], Ctoronto_merged['Longitude'], Ctoronto_merged['Neighborhood'], Ctoronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

This notebook is part of a course on **Coursera** called *Applied Data Science Capstone*